In [11]:
# MAX_ATTEMPTS = 1
# START_RUN_INDEX = 0
# RUN_NOTE = "Second run with code agent only"
# RUN_AGENTS = ['code', 'github', 'docs', 'git']

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import re
import os
import sys
import time
BASE_DIR = os.path.abspath(os.path.abspath("../"))
sys.path.append(BASE_DIR)

In [14]:
import pandas as pd
from langchain_core.messages import BaseMessage, HumanMessage
from src.orchestration.graph import graph
from src.utils import safe_get_content, remove_think_block

In [15]:
questions = pd.read_csv(BASE_DIR+"/data/external/filled_questions_v2.csv", index_col='id')
questions

,project,question_id,customized_quesstion
id,,,
1,wg/scrypt,1,Is there an entity named something like SCrypt...
2,wg/scrypt,2,Is there an entity named something like packag...
3,wg/scrypt,3,Is there an entity named something like loadSi...
4,wg/scrypt,4,Is there an entity named something like PBKDFT...
5,wg/scrypt,5,Is there an entity named something like blockm...
...,...,...,...
166,pedrovgs/Renderers,30,What is the difference between these similar p...
167,pedrovgs/Renderers,31,How can we know this object RendererTest has b...
168,pedrovgs/Renderers,32,"How are these types or objects related (e.g., ..."


In [16]:
projects = pd.read_csv(BASE_DIR+"/data/processed/projects.csv")
projects["repo_path"] = projects["repo"].apply(lambda x: f"{BASE_DIR}/data/processed/{x.split('/')[-1]}")

In [17]:
projects.head(5)

,code_db,repo,docs_url,repo_path
0,scrypt.db,wg/scrypt,'https://www.tarsnap.com/scrypt/scrypt.pdf',/Users/yoseph/Study/VT/Course Work/Spring 25/C...
1,groovy-core.db,groovy/groovy-core,'https://groovy-lang.org/documentation.html',/Users/yoseph/Study/VT/Course Work/Spring 25/C...
2,natty.db,joestelmach/natty,'',/Users/yoseph/Study/VT/Course Work/Spring 25/C...
3,yahoofinance-api.db,sstrickx/yahoofinance-api,'https://financequotes-api.com/',/Users/yoseph/Study/VT/Course Work/Spring 25/C...
4,Renderers.db,pedrovgs/Renderers,'',/Users/yoseph/Study/VT/Course Work/Spring 25/C...


In [18]:
data = pd.merge(questions, projects, left_on='project', right_on='repo', how='inner')
data.head(5)

,project,question_id,customized_quesstion,code_db,repo,docs_url,repo_path
0,wg/scrypt,1,Is there an entity named something like SCrypt...,scrypt.db,wg/scrypt,'https://www.tarsnap.com/scrypt/scrypt.pdf',/Users/yoseph/Study/VT/Course Work/Spring 25/C...
1,wg/scrypt,2,Is there an entity named something like packag...,scrypt.db,wg/scrypt,'https://www.tarsnap.com/scrypt/scrypt.pdf',/Users/yoseph/Study/VT/Course Work/Spring 25/C...
2,wg/scrypt,3,Is there an entity named something like loadSi...,scrypt.db,wg/scrypt,'https://www.tarsnap.com/scrypt/scrypt.pdf',/Users/yoseph/Study/VT/Course Work/Spring 25/C...
3,wg/scrypt,4,Is there an entity named something like PBKDFT...,scrypt.db,wg/scrypt,'https://www.tarsnap.com/scrypt/scrypt.pdf',/Users/yoseph/Study/VT/Course Work/Spring 25/C...
4,wg/scrypt,5,Is there an entity named something like blockm...,scrypt.db,wg/scrypt,'https://www.tarsnap.com/scrypt/scrypt.pdf',/Users/yoseph/Study/VT/Course Work/Spring 25/C...


In [19]:
runs = pd.read_csv("../data/external/runs.csv")
runs

,run_id,note,git,github,code,docs
0,9,Second round run with all agents,True,True,True,True
1,10,Second round run with all agents except docs,True,True,True,False
2,11,Second round run with code and git agents,True,False,True,False
3,12,Second round run with code and github agents,False,True,True,False
4,13,Second round run with code agnet only,False,False,True,False
5,14,Second round code and docs agents only,False,False,True,True


In [20]:
for _, row in runs.iterrows():
    RUN_ID = row['run_id']
    RUN_NOTE = row['note']
    RUN_GIT = row['git']
    RUN_GITHUB = row['github']
    RUN_CODE = row['code']
    RUN_DOCS = row['docs']
    start = time.time()
    generated = []
    for i, row in data.iterrows():
        question = row['customized_quesstion']
        # result = graph.invoke
        code_db = BASE_DIR+"/data/processed/"+row['code_db']
        repository_path = row['repo_path']
        github_id = row['repo']
        docs_url = row['docs_url']
        attempt = 0
            # response = f"Response for question '{question}' on {repository_path} with code_db {code_db} and github_id {github_id} running agents: {RUN_AGENTS}"
        try: 
            result = graph.invoke({
                "user_query": [HumanMessage(question)],
                "source_db": code_db,
                "repository_path": repository_path,
                "docs_source": docs_url,
                "github_url": github_id,
                "run_git": RUN_GIT,
                "run_github": RUN_GITHUB,
                "run_code":RUN_CODE,
                "run_docs": RUN_DOCS
                })
            if(result["final_response"]):
                final_response = result['final_response'].content
            else:
                final_response = 'Error: cound not answer'
            
        except Exception as e:
            print(f"Error: {e}")
            final_response = "Error: cound not answer"
        
        generated.append({
            "run_id": RUN_ID,
            "question_id": row["question_id"],
            "question": question,
            "project": github_id,
            "final_response": final_response,
        })  
        generated_df = pd.DataFrame(generated)
        generated_df.to_csv(f"{BASE_DIR}/data/generated/generated_{RUN_ID}.csv", index=False)
    end = time.time()
    with open(f"{BASE_DIR}/data/generated/completed_runs.csv", "a") as f:
        f.write(f"{RUN_ID},{len(generated_df)},{end - start:.4f}\n")

Error: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT
Error: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT
Error: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT
Error: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT


In [21]:
generated_df = pd.DataFrame(generated)

In [22]:
generated_df.head()

,run_id,question_id,question,project,final_response
0,14,1,Is there an entity named something like SCrypt...,wg/scrypt,"\n\nBased on this information, there doesn't a..."
1,14,2,Is there an entity named something like packag...,wg/scrypt,"\n\nBased on the limited information provided,..."
2,14,3,Is there an entity named something like loadSi...,wg/scrypt,Error: cound not answer
3,14,4,Is there an entity named something like PBKDFT...,wg/scrypt,"\n\nYes, there is an entity named PBKDFTest in..."
4,14,5,Is there an entity named something like blockm...,wg/scrypt,Error: cound not answer


In [23]:
generated_df.to_csv(f"{BASE_DIR}/data/generated/generated_{RUN_ID}.csv", index=False)

In [24]:

# Get current formatted time
